<a href="https://colab.research.google.com/github/JunyeChen2025/Camera_Privacy_Shield/blob/main/Face_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -q '/content/drive/My Drive/YOLO_Data/Wider_Face_Dataset.zip' -d /content/Dataset

In [3]:
import os
import shutil

# Set Path
base_path = "/content/Dataset/Wider_Face_Dataset"
output_path = os.path.join(base_path, "images")
os.makedirs(output_path, exist_ok=True)

# Define the mapping of source directories to target subfolders
folder_map = {
    "WIDER_train": "train",
    "WIDER_val": "val",
    "WIDER_test": "test"
}

for src_folder, target_subfolder in folder_map.items():
    src_path = os.path.join(base_path, src_folder)
    dst_path = os.path.join(output_path, target_subfolder)
    os.makedirs(dst_path, exist_ok=True)

    # Iterate over subdirectories
    for root, dirs, files in os.walk(src_path):
        for file in files:
            if file.lower().endswith(".jpg"):
                src_file_path = os.path.join(root, file)
                dst_file_path = os.path.join(dst_path, file)
                shutil.copy2(src_file_path, dst_file_path)

    print(f"Extraction completed：{src_folder} -> images/{target_subfolder}")

# Delete original folders
for folder in folder_map.keys():
    folder_path = os.path.join(base_path, folder)
    shutil.rmtree(folder_path)
    print(f"Deleted orginal directory：{folder_path}")

print("All images have been collated.")


Extraction completed：WIDER_train -> images/train
Extraction completed：WIDER_val -> images/val
Extraction completed：WIDER_test -> images/test
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_train
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_val
Deleted orginal directory：/content/Dataset/Wider_Face_Dataset/WIDER_test
All images have been collated.


In [ ]:
import os
from PIL import Image

def convert_widerface_to_yolo(txt_path, image_base_dir, save_label_dir, class_id=0):
    os.makedirs(save_label_dir, exist_ok=True)

    with open(txt_path, 'r') as f:
        lines = f.readlines()

    idx = 0
    while idx < len(lines):
        filename = lines[idx].strip()
        idx += 1

        if idx >= len(lines):
            break

        try:
            face_count = int(lines[idx].strip())
        except ValueError:
            print(f"Skipping invalid face count at line {idx}: {lines[idx]}")
            idx += 1
            continue
        idx += 1

        label_lines = []

        image_path = os.path.join(image_base_dir, os.path.basename(filename))
        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}, skipping.")
            idx += face_count
            continue

        try:
            img = Image.open(image_path)
            w_img, h_img = img.size
        except:
            print(f"Failed to open image: {image_path}")
            idx += face_count
            continue

        for _ in range(face_count):
            if idx >= len(lines):
                break
            parts = lines[idx].strip().split()
            if len(parts) < 4:
                idx += 1
                continue

            try:
                x, y, w, h = map(float, parts[:4])
                x_center = (x + w / 2) / w_img
                y_center = (y + h / 2) / h_img
                w_norm = w / w_img
                h_norm = h / h_img
                label_line = f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"
                label_lines.append(label_line)
            except:
                pass
            idx += 1

        # Save label
        base_name = os.path.splitext(os.path.basename(filename))[0]
        label_file = os.path.join(save_label_dir, base_name + '.txt')
        with open(label_file, 'w') as f:
            f.write('\n'.join(label_lines))


In [ ]:
# Set Path
base_path = "/content/Dataset/Wider_Face_Dataset"
anno_path = os.path.join(base_path, "Annotations")
image_dir = os.path.join(base_path, "images")

# Convert training set and validation set
convert_widerface_to_yolo(
    txt_path=os.path.join(anno_path, "wider_face_train_bbx_gt.txt"),
    image_base_dir=os.path.join(image_dir, "train"),
    save_label_dir=os.path.join(base_path, "labels/train")
)

convert_widerface_to_yolo(
    txt_path=os.path.join(anno_path, "wider_face_val_bbx_gt.txt"),
    image_base_dir=os.path.join(image_dir, "val"),
    save_label_dir=os.path.join(base_path, "labels/val")
)


Skipping invalid face count at line 10424: 0--Parade/0_Parade_Parade_0_630.jpg

Skipping invalid face count at line 10426: 20 186 14 20 1 0 0 0 0 0 

Skipping invalid face count at line 10428: 119 181 14 17 2 0 0 0 2 0 

Skipping invalid face count at line 10430: 375 129 46 62 0 0 0 0 0 0 

Skipping invalid face count at line 10432: 349 122 26 38 0 0 0 0 0 0 

Skipping invalid face count at line 10434: 527 174 10 16 1 0 0 0 0 0 

Skipping invalid face count at line 10436: 695 143 49 64 0 0 0 0 0 0 

Skipping invalid face count at line 10438: 822 138 29 38 0 0 0 0 2 0 

Skipping invalid face count at line 10440: 954 156 12 18 1 0 0 0 0 0 

Skipping invalid face count at line 10442: 615 143 22 24 1 0 0 0 2 0 

Skipping invalid face count at line 86539: 2--Demonstration/2_Demonstration_Political_Rally_2_414.jpg

Skipping invalid face count at line 86541: 52 130 120 204 0 0 0 0 0 1 

Skipping invalid face count at line 86543: 400 156 62 64 1 0 0 0 1 1 

Skipping invalid face count at line 

In [4]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model.train(
    data="/content/Dataset/Wider_Face_Dataset/wider_face.yaml",
    epochs=30,
    imgsz=640,
    batch=16,
    name="face_yolov8"
)

Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Dataset/Wider_Face_Dataset/wider_face.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=face_yolov83, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, p

100%|██████████| 755k/755k [00:00<00:00, 24.4MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

YOLOv8n summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 80.0MB/s]


...

      30/30      14.4G      1.416     0.7188      1.069         69        640: 100%|██████████| 805/805 [04:09<00:00,  3.22it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:34<00:00,  2.97it/s]


                   all       3226      39707      0.809      0.522      0.597       0.32

30 epochs completed in 2.455 hours.
Optimizer stripped from runs/detect/face_yolov83/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/face_yolov83/weights/best.pt, 6.2MB

Validating runs/detect/face_yolov83/weights/best.pt...
Ultralytics 8.3.167 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 101/101 [00:33<00:00,  3.01it/s]


                   all       3226      39707       0.81      0.522      0.597       0.32
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/face_yolov83


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fa554d5c1d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
!zip -r runs_face_yolov8.zip runs

from google.colab import files
files.download('runs_face_yolov8.zip')


  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/face_yolov8/ (stored 0%)
  adding: runs/detect/face_yolov8/weights/ (stored 0%)
  adding: runs/detect/face_yolov8/args.yaml (deflated 53%)
  adding: runs/detect/face_yolov82/ (stored 0%)
  adding: runs/detect/face_yolov82/weights/ (stored 0%)
  adding: runs/detect/face_yolov82/args.yaml (deflated 53%)
  adding: runs/detect/face_yolov83/ (stored 0%)
  adding: runs/detect/face_yolov83/BoxF1_curve.png (deflated 17%)
  adding: runs/detect/face_yolov83/train_batch16101.jpg (deflated 12%)
  adding: runs/detect/face_yolov83/confusion_matrix_normalized.png (deflated 36%)
  adding: runs/detect/face_yolov83/val_batch1_labels.jpg (deflated 7%)
  adding: runs/detect/face_yolov83/labels_correlogram.jpg (deflated 33%)
  adding: runs/detect/face_yolov83/BoxPR_curve.png (deflated 20%)
  adding: runs/detect/face_yolov83/labels.jpg (deflated 35%)
  adding: runs/detect/face_yolov83/train_batch1.jpg (deflated 2%)
  adding

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
from ultralytics import YOLO

# Upload existing model best.py
model = YOLO("/content/best.pt")

# Perform reasoning and save results
model.predict(
    source="/content/Dataset/Wider_Face_Dataset/images/test",
    save=True,
    save_txt=True,
    imgsz=640,
    conf=0.25,
    name="face_test_predictions"
)


FileNotFoundError: No images or videos found in /content/Dataset/Wider_Face_Dataset/images/test. Supported formats are:
images: {'tif', 'heic', 'png', 'jpeg', 'dng', 'jpg', 'tiff', 'pfm', 'webp', 'mpo', 'bmp'}
videos: {'asf', 'avi', 'mov', 'ts', 'gif', 'mkv', 'wmv', 'mpg', 'mpeg', 'webm', 'm4v', 'mp4'}

In [ ]:
from google.colab import files

!zip -r face_test_predictions.zip runs/detect/face_test_predictions
files.download("face_test_predictions.zip")


updating: runs/detect/face_test_predictions/ (stored 0%)
updating: runs/detect/face_test_predictions/bus.jpg (deflated 7%)
updating: runs/detect/face_test_predictions/zidane.jpg (deflated 9%)
updating: runs/detect/face_test_predictions/labels/ (stored 0%)
updating: runs/detect/face_test_predictions/labels/bus.txt (deflated 29%)
updating: runs/detect/face_test_predictions/labels/zidane.txt (deflated 27%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
from ultralytics import YOLO
import os
from glob import glob
import shutil
from zipfile import ZipFile

# Prepare reasoning pictures
source_dir = "/content/Dataset/Wider_Face_Dataset/images/test"
subset_dir = "/content/test_subset"

# Re-create the subset catalogue
if os.path.exists(subset_dir):
    shutil.rmtree(subset_dir)
os.makedirs(subset_dir, exist_ok=True)

# Access first 5,000 jpg images
image_files = sorted(glob(os.path.join(source_dir, "*.jpg")))[:5000]
for img_path in image_files:
    shutil.copy(img_path, subset_dir)

print(f"✅ Copied {len(image_files)} images to test subset")

# Load model and start reasoning
model = YOLO("/content/best.pt")
model.predict(
    source=subset_dir,
    save=True,
    save_txt=True,
    imgsz=640,
    conf=0.25,
    name="face_test_5000_predictions"
)

# Compression results catalogue
prediction_dir = "runs/detect/face_test_5000_predictions"
zip_path = "/content/face_test_5000_predictions.zip"

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', prediction_dir)
print("Prediction results zipped.")

# Download results and save locally
from google.colab import files
files.download(zip_path)


Streaming output truncated to the last 5000 lines.
image 5/5000 /content/test_subset/0_Parade_Parade_0_1025.jpg: 480x640 1 face, 7.4ms
image 6/5000 /content/test_subset/0_Parade_Parade_0_103.jpg: 640x480 6 faces, 32.0ms
image 7/5000 /content/test_subset/0_Parade_Parade_0_1034.jpg: 480x640 5 faces, 8.6ms
image 8/5000 /content/test_subset/0_Parade_Parade_0_1039.jpg: 480x640 7 faces, 6.1ms
image 9/5000 /content/test_subset/0_Parade_Parade_0_1046.jpg: 608x640 1 face, 34.8ms
image 10/5000 /content/test_subset/0_Parade_Parade_0_1047.jpg: 480x640 7 faces, 6.9ms
image 11/5000 /content/test_subset/0_Parade_Parade_0_107.jpg: 448x640 162 faces, 33.2ms
image 12/5000 /content/test_subset/0_Parade_Parade_0_111.jpg: 448x640 58 faces, 6.3ms
image 13/5000 /content/test_subset/0_Parade_Parade_0_112.jpg: 480x640 1 face, 7.2ms
image 14/5000 /content/test_subset/0_Parade_Parade_0_114.jpg: 448x640 10 faces, 6.7ms
image 15/5000 /content/test_subset/0_Parade_Parade_0_115.jpg: 480x640 8 faces, 6.6ms
image 16/5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>